In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
%matplotlib inline
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [3]:
import matplotlib.pylab as plt
# https://docs.scipy.org/doc/numpy/reference/routines.math.html
import numpy as np

In [4]:
from datetime import tzinfo, timedelta, datetime

In [5]:
from distutils.version import StrictVersion

In [6]:
import sklearn

assert StrictVersion(sklearn.__version__ ) >= StrictVersion('0.18.1')

sklearn.__version__

'0.18.1'

In [7]:
import tensorflow as tf
tf.logging.set_verbosity(tf.logging.ERROR)

assert StrictVersion(tf.__version__) >= StrictVersion('1.1.0')

tf.__version__

'1.2.1'

In [8]:
import keras

assert StrictVersion(keras.__version__) >= StrictVersion('2.0.0')

keras.__version__

Using TensorFlow backend.


'2.0.6'

In [9]:
# !curl -O https://raw.githubusercontent.com/DJCordhose/speed-limit-signs/master/data/speed-limit-signs.zip
# !curl -O https://raw.githubusercontent.com/DJCordhose/speed-limit-signs/master/data/augmented-signs.zip    

In [10]:
# https://docs.python.org/3/library/zipfile.html
# from zipfile import ZipFile
# zip = ZipFile(r'speed-limit-signs.zip')
# zip.extractall('.')
# zip = ZipFile(r'augmented-signs.zip')
# zip.extractall('.')

In [11]:
# !ls -l speed-limit-signs

In [12]:
# !ls -l augmented-signs

In [13]:
import os
import skimage.data
import skimage.transform
from keras.utils.np_utils import to_categorical
import numpy as np

def load_data(data_dir, type=".ppm"):
    num_categories = 6

    # Get all subdirectories of data_dir. Each represents a label.
    directories = [d for d in os.listdir(data_dir) 
                   if os.path.isdir(os.path.join(data_dir, d))]
    # Loop through the label directories and collect the data in
    # two lists, labels and images.
    labels = []
    images = []
    for d in directories:
        label_dir = os.path.join(data_dir, d)
        file_names = [os.path.join(label_dir, f) for f in os.listdir(label_dir) if f.endswith(type)]
        # For each label, load it's images and add them to the images list.
        # And add the label number (i.e. directory name) to the labels list.
        for f in file_names:
            images.append(skimage.data.imread(f))
            labels.append(int(d))
    images64 = [skimage.transform.resize(image, (64, 64)) for image in images]
    y = np.array(labels)
    y = to_categorical(y, num_categories)
    X = np.array(images64)
    return X, y

In [33]:
# Depends on harware GPU architecture, set as high as possible (this works well on K80)
BATCH_SIZE = 500

In [14]:
# Load datasets.
ROOT_PATH = "./"

In [15]:
original_dir = os.path.join(ROOT_PATH, "speed-limit-signs")
original_images, original_labels = load_data(original_dir, type=".ppm")

In [16]:
data_dir = os.path.join(ROOT_PATH, "augmented-signs")
augmented_images, augmented_labels = load_data(data_dir, type=".png")

In [40]:
# Turn this around if you want the large training set using augmented data or the original one

# X, y = original_images, original_labels
X, y = augmented_images, augmented_labels

In [41]:
# Same as above

# evaluation_X, evaluation_y = augmented_images, augmented_labels
evaluation_X, evaluation_y = original_images, original_labels

In [19]:
from sklearn.model_selection import train_test_split

In [20]:
checkpoint_callback = keras.callbacks.ModelCheckpoint('../tmp/model-checkpoints/weights.epoch-{epoch:02d}-val_loss-{val_loss:.2f}.hdf5');

In [42]:
early_stopping_callback = keras.callbacks.EarlyStopping(monitor='val_loss', patience=30, verbose=1)

In [43]:
# https://github.com/tensorflow/tensorflow/blob/master/tensorflow/tensorboard/README.md
# https://keras.io/callbacks/#tensorboard
# http://stackoverflow.com/questions/42112260/how-do-i-use-the-tensorboard-callback-of-keras
tb_callback = keras.callbacks.TensorBoard(log_dir='../tmp/tf_log')
#                                          histogram_freq=1, write_graph=True, write_images=True)
#                                          histogram_freq=1, write_graph=True, write_images=True)
# tbCallBack = keras.callbacks.TensorBoard(log_dir='./logs', histogram_freq=0, batch_size=32, write_graph=True, write_grads=False, write_images=False, embeddings_freq=0, embeddings_layer_names=None, embeddings_metadata=None)
# To start tensorboard
# tensorboard --logdir=/mnt/c/Users/olive/Development/ml/tf_log
# open http://localhost:6006

In [44]:
# we want to distribute our different classes equally over test and train, this works using stratify
# https://github.com/amueller/scipy-2017-sklearn/blob/master/notebooks/04.Training_and_Testing_Data.ipynb
# http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y, )

In [45]:
X_train.shape, y_train.shape

((3032, 64, 64, 3), (3032, 6))

In [46]:
from keras.models import Model
from keras.layers import Dense, Dropout, Activation, Flatten, Input
from keras.layers import Convolution2D, MaxPooling2D

# drop_out = 0.9
# drop_out = 0.75
drop_out = 0.5
# drop_out = 0.25
# drop_out = 0.0

# input tensor for a 3-channel 64x64 image
inputs = Input(shape=(64, 64, 3))

# one block of convolutional layers
x = Convolution2D(64, 3, 3, activation='relu')(inputs)
# x = Dropout(drop_out)(x)
x = Convolution2D(64, 3, 3, activation='relu')(x)
# x = Dropout(drop_out)(x)
x = Convolution2D(64, 3, 3, activation='relu')(x)
x = MaxPooling2D(pool_size=(2, 2))(x)
x = Dropout(drop_out)(x)

# one more block
x = Convolution2D(128, 3, 3, activation='relu')(x)
# x = Dropout(drop_out)(x)
x = Convolution2D(128, 3, 3, activation='relu')(x)
x = MaxPooling2D(pool_size=(2, 2))(x)
x = Dropout(drop_out)(x)

# one more block
x = Convolution2D(256, 3, 3, activation='relu')(x)
x = MaxPooling2D(pool_size=(2, 2))(x)
x = Dropout(drop_out)(x)

x = Flatten()(x)
x = Dense(256, activation='relu')(x)
x = Dropout(drop_out)(x)

# softmax activation, 6 categories
predictions = Dense(6, activation='softmax')(x)
model = Model(input=inputs, output=predictions)
model.summary()
model.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         (None, 64, 64, 3)         0         
_________________________________________________________________
conv2d_13 (Conv2D)           (None, 62, 62, 64)        1792      
_________________________________________________________________
conv2d_14 (Conv2D)           (None, 60, 60, 64)        36928     
_________________________________________________________________
conv2d_15 (Conv2D)           (None, 58, 58, 64)        36928     
_________________________________________________________________
max_pooling2d_7 (MaxPooling2 (None, 29, 29, 64)        0         
_________________________________________________________________
dropout_9 (Dropout)          (None, 29, 29, 64)        0         
_________________________________________________________________
conv2d_16 (Conv2D)           (None, 27, 27, 128)       73856     
__________

In [47]:
!rm -rf ../tmp/tf_log
!rm -rf ../tmp/model-checkpoints

!mkdir ../tmp/model-checkpoints
!mkdir ../tmp/tf_log

In [ ]:
# Running on a GPU bach size might be critical depdendng on the GPU memory available
# more is desirable, but we might end up using 50 only 
print(datetime.utcnow().isoformat())
# BE CAREFUL, validation data is always the last data sets and not shuffled
# https://keras.io/getting-started/faq/#how-is-the-validation-split-computed
model.fit(X_train, y_train, epochs=1000, batch_size=BATCH_SIZE, validation_split=0.3, 
#           callbacks=[tb_callback, early_stopping_callback])
          callbacks=[tb_callback])
# model.fit(X_train, y_train, epochs=50, batch_size=200, validation_split=0.3)
print(datetime.utcnow().isoformat())

2017-07-31T08:01:02.689115
Train on 2122 samples, validate on 910 samples
Epoch 1/1000
2122/2122 [==============================] - 11s - loss: 2.0733 - acc: 0.1786 - val_loss: 1.7863 - val_acc: 0.1714
Epoch 2/1000
2122/2122 [==============================] - 5s - loss: 1.7719 - acc: 0.1989 - val_loss: 1.7685 - val_acc: 0.2143
Epoch 3/1000
2122/2122 [==============================] - 5s - loss: 1.7892 - acc: 0.2092 - val_loss: 1.7748 - val_acc: 0.2143
Epoch 4/1000
2122/2122 [==============================] - 5s - loss: 1.7691 - acc: 0.2022 - val_loss: 1.7801 - val_acc: 0.2143
Epoch 5/1000
2122/2122 [==============================] - 5s - loss: 1.7605 - acc: 0.2031 - val_loss: 1.7649 - val_acc: 0.2121
Epoch 6/1000
2122/2122 [==============================] - 5s - loss: 1.7514 - acc: 0.2050 - val_loss: 1.7859 - val_acc: 0.1714
Epoch 7/1000
2122/2122 [==============================] - 5s - loss: 1.7761 - acc: 0.1956 - val_loss: 1.7741 - val_acc: 0.2121
Epoch 8/1000
2122/2122 [============

2122/2122 [==============================] - 5s - loss: 1.1642 - acc: 0.6692 - val_loss: 1.5434 - val_acc: 0.4714
Epoch 128/1000
2122/2122 [==============================] - 5s - loss: 0.6588 - acc: 0.7955 - val_loss: 0.8177 - val_acc: 0.7451
Epoch 129/1000
2122/2122 [==============================] - 5s - loss: 0.2091 - acc: 0.9321 - val_loss: 0.8719 - val_acc: 0.7681
Epoch 130/1000
2122/2122 [==============================] - 5s - loss: 0.1785 - acc: 0.9378 - val_loss: 0.9471 - val_acc: 0.7626
Epoch 131/1000
2122/2122 [==============================] - 5s - loss: 0.1768 - acc: 0.9345 - val_loss: 1.1168 - val_acc: 0.7220
Epoch 132/1000
2122/2122 [==============================] - 5s - loss: 0.2881 - acc: 0.9025 - val_loss: 1.0266 - val_acc: 0.7626
Epoch 133/1000
2122/2122 [==============================] - 5s - loss: 0.2407 - acc: 0.9156 - val_loss: 0.9112 - val_acc: 0.7769
Epoch 134/1000
2122/2122 [==============================] - 5s - loss: 0.7425 - acc: 0.7705 - val_loss: 0.9645 -

2122/2122 [==============================] - 4s - loss: 0.0317 - acc: 0.9925 - val_loss: 0.6328 - val_acc: 0.8857
Epoch 727/1000
2122/2122 [==============================] - 4s - loss: 0.0279 - acc: 0.9929 - val_loss: 0.6653 - val_acc: 0.8780
Epoch 728/1000
2122/2122 [==============================] - 4s - loss: 0.0181 - acc: 0.9953 - val_loss: 0.6554 - val_acc: 0.8901
Epoch 729/1000
2122/2122 [==============================] - 4s - loss: 0.0385 - acc: 0.9929 - val_loss: 0.7656 - val_acc: 0.8857
Epoch 730/1000
2122/2122 [==============================] - 4s - loss: 0.1185 - acc: 0.9727 - val_loss: 0.7800 - val_acc: 0.8813
Epoch 731/1000
2122/2122 [==============================] - 4s - loss: 0.0288 - acc: 0.9929 - val_loss: 0.7477 - val_acc: 0.8813
Epoch 732/1000
2122/2122 [==============================] - 4s - loss: 0.0798 - acc: 0.9826 - val_loss: 0.6671 - val_acc: 0.8527
Epoch 733/1000
2122/2122 [==============================] - 4s - loss: 0.0794 - acc: 0.9811 - val_loss: 0.5057 -

2122/2122 [==============================] - 4s - loss: 0.0129 - acc: 0.9967 - val_loss: 0.6570 - val_acc: 0.8967
Epoch 853/1000
2122/2122 [==============================] - 4s - loss: 0.0195 - acc: 0.9958 - val_loss: 0.6402 - val_acc: 0.9055
Epoch 854/1000
2122/2122 [==============================] - 4s - loss: 0.0239 - acc: 0.9943 - val_loss: 0.8484 - val_acc: 0.8330
Epoch 855/1000
2122/2122 [==============================] - 5s - loss: 0.2936 - acc: 0.9406 - val_loss: 0.4993 - val_acc: 0.8813
Epoch 856/1000
2122/2122 [==============================] - 4s - loss: 0.0546 - acc: 0.9859 - val_loss: 0.6301 - val_acc: 0.8989
Epoch 857/1000
2122/2122 [==============================] - 4s - loss: 0.0336 - acc: 0.9939 - val_loss: 0.7225 - val_acc: 0.8945
Epoch 858/1000
2122/2122 [==============================] - 4s - loss: 0.0512 - acc: 0.9882 - val_loss: 0.6017 - val_acc: 0.8879
Epoch 859/1000
2122/2122 [==============================] - 4s - loss: 0.0606 - acc: 0.9873 - val_loss: 0.4461 -

2122/2122 [==============================] - 4s - loss: 0.0345 - acc: 0.9929 - val_loss: 0.7063 - val_acc: 0.8769
Epoch 979/1000
2122/2122 [==============================] - 5s - loss: 0.0625 - acc: 0.9892 - val_loss: 0.6148 - val_acc: 0.9066
Epoch 980/1000
2122/2122 [==============================] - 4s - loss: 0.0247 - acc: 0.9939 - val_loss: 0.6866 - val_acc: 0.8945
Epoch 981/1000
2122/2122 [==============================] - 4s - loss: 0.0265 - acc: 0.9943 - val_loss: 0.6597 - val_acc: 0.9055
Epoch 982/1000
2122/2122 [==============================] - 4s - loss: 0.0367 - acc: 0.9934 - val_loss: 0.7127 - val_acc: 0.8835
Epoch 983/1000
2122/2122 [==============================] - 4s - loss: 0.0843 - acc: 0.9859 - val_loss: 0.6169 - val_acc: 0.8835
Epoch 984/1000
2122/2122 [==============================] - 4s - loss: 0.0512 - acc: 0.9896 - val_loss: 0.5646 - val_acc: 0.8989
Epoch 985/1000
2122/2122 [==============================] - 4s - loss: 0.0442 - acc: 0.9920 - val_loss: 0.7267 -

In [34]:
train_loss, train_accuracy = model.evaluate(X_train, y_train, batch_size=500)
train_loss, train_accuracy

303/303 [==============================] - 1s


(0.072976924479007721, 0.99009901285171509)

In [35]:
test_loss, test_accuracy = model.evaluate(X_test, y_test, batch_size=BATCH_SIZE)
test_loss, test_accuracy

76/76 [==============================] - 0s


(0.12756867706775665, 0.98684209585189819)

In [36]:
evaluation_loss, evaluation_accuracy = model.evaluate(evaluation_X, evaluation_y, batch_size=BATCH_SIZE)
evaluation_loss, evaluation_accuracy

3790/3790 [==============================] - 5s     


(11.690376923392504, 0.21820580361700939)

In [37]:
!mkdir models

mkdir: cannot create directory ‘models’: File exists


In [38]:
# model.save('models/conv-vgg-augmented.hdf5')
model.save('models/conv-vgg-original.hdf5')

In [39]:
!ls -lh models

total 35M
-rw-rw-r-- 1 ubuntu ubuntu 18M Jul 21 19:56 conv-vgg-augmented.hdf5
-rw-rw-r-- 1 ubuntu ubuntu 18M Jul 31 07:54 conv-vgg-original.hdf5


In [33]:
# !curl --upload-file ./models/conv-vgg-augmented.hdf5 https://transfer.sh/conv-vgg-augmented.hdf5
!curl --upload-file ./models/conv-vgg-original.hdf5 https://transfer.sh/conv-vgg-original.hdf5

https://transfer.sh/K5RH8/conv-vgg-augmented.hdf5